# Lorentz Booster
- Generate a whole bunch of particles moving at some relative velocity, then lorentz boost into their frame. This is done compactly by
$$
\bar{\eta}^\mu = \Lambda^\mu_\nu \eta^\nu
$$
where $\eta$ is the 

In [1]:
from enum import Enum
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm
%matplotlib inline

# my own little library for dealing with four vectors
import fourvectors

In [2]:
### Constants
c = 3E8

### Create enums of particle properties, could be useful for multi-species experiments
class Proton(Enum):
    NAME   = "PROTON"
    MASS   = 1.672E-27 #kg
    CHARGE = 1.602E-19 #C

class Electron(Enum):
    NAME   = "ELECTRON"
    MASS   = 9.109E-21 #kg
    CHARGE =-1.602E-19 #C

class Reference(Enum):
    """A particle that is simply here to be a reference point."""
    NAME   = "REFERENCE PARTICLE"
    MASS   = 0        #kg
    CHARGE = 0        #C

In [ ]:
### Create the particle class, this holds all of the nice info about each particle
class Particle():
    def __init__(
            self,
            species,
            pos  = None,  # a four vector
            v0   = None,  # a three-velocity
            eta0 = None,  # a four-velocity
            frame= "LAB"  # defaults to lab frame
        ):

        # if no initial three-velocity specified, assume 0
        if v0 is None:
            eta0 = np.array([0, 0, 0, 0])
        else:
            # otherwise set the four velocity to match it
            eta0 = fourvectors.to_four_velocity(v0)

        # if no initial four-velocity specified, assume 0
        if eta0 is None:
            eta0 = np.array([0, 0, 0, 0])

        # if no position specified, assume origin
        if pos is None:
            pos = np.array([0, 0, 0, 0])

        # Get the parameters from the species of particle
        self.name = species.NAME.value
        self.mass = species.MASS.value
        self.charge = species.CHARGE.value

        self.eta = eta0
        self.frame = frame

    ### Lorentz Boost Functionality
    def lorentz_boost_to(self, other):
        """Lorentz boost to another particle's frame"""

        # if the frame is lab, then we're boosting to another particle
        if self.frame == "LAB":
            self.frame = other.name
        else:
            self.frame = other.frame

        self.lorentz_boost_from_4v(other.eta)

    def lorentz_boost_from_4v(self, to_eta):
        """Lorentz boost given a four-velocity"""
        self.eta = np.linalg.matmul(fourvectors.lorentz_matrix_x_4v(to_eta), self.eta)

    def lorentz_boost_from_3v(self, to_v):
        """Lorentz boost given a three-velocity"""
        self.lorentz_boost_from_4v(fourvectors.to_four_velocity(to_v))



    ### Getters
    def get_3v(self):
        return fourvectors.to_three_velocity(self.eta)

In [44]:
reference = Particle(Reference, v0 = np.array([2E8, 0, 0]))
other_particle = Particle(Electron, v0 = np.array([2.2E8, 0, 0]))

np.set_printoptions(formatter={'float': '{:.5e}'.format})

print(f"--- {reference.frame} FRAME ---")
print(f"{reference.name} Velocity: {reference.get_3v()}")
print(f"{other_particle.name} Velocity: {other_particle.get_3v()}")

# perform the lorentz boost
other_particle.lorentz_boost_to(reference)
reference.lorentz_boost_to(reference)

print(f"--- {reference.frame} FRAME ---")
print(f"{reference.name} Velocity: {reference.get_3v()}")
print(f"{other_particle.name} Velocity: {other_particle.get_3v()}")

--- LAB FRAME ---
REFERENCE PARTICLE Velocity: [[2.00000e+08 0.00000e+00 0.00000e+00]]
ELECTRON Velocity: [[2.20000e+08 0.00000e+00 0.00000e+00]]
--- REFERENCE PARTICLE FRAME ---
REFERENCE PARTICLE Velocity: [[0.00000e+00 0.00000e+00 0.00000e+00]]
ELECTRON Velocity: [[3.91304e+07 0.00000e+00 0.00000e+00]]
